In [ ]:
from comparison_utils import get_ride_data, analyze_and_plot_ride_data

In [ ]:
SUMO_SIM_DATA_FOLDER = "./sim_data"

In [ ]:
comparison_data = [
    {
        "scenario_name": "Frankfurter Allee (Alfredstr. to Ruschestr.)",
        "sumo_sim_files": ["B1_new_params.csv", "B1_default.csv"],
        "start_rect_coords": (13.489109,52.512004,13.48953,52.512492),
        "end_rect_coords": (13.484732,52.512494,13.485153,52.512982)
    },
    {
        "scenario_name": "Frankfurter Allee (Rathausstr. to Möllendorfstr.)",
        "sumo_sim_files": ["B1_new_params.csv", "B1_default.csv"],
        "start_rect_coords": (13.481105,52.513004,13.481526,52.513374),
        "end_rect_coords": (13.477682,52.513305,13.478103,52.513675)
    },
    {
        "scenario_name": "Einsteinufer (Marchbrücke to Charlotenburger Brücke)",
        "sumo_sim_files": ["CHB_new_params.csv", "CHB_default.csv"],
        "start_rect_coords": (13.324764,52.516524,13.32511,52.517017),
        "end_rect_coords": (13.329785,52.513501,13.330475,52.513615)
    },
    {
        "scenario_name": "Einsteinufer (Charlotenburger Brücke to Hardenbergstr.)",
        "sumo_sim_files": ["CHB_new_params.csv", "CHB_default.csv"],
        "start_rect_coords": (13.330096,52.51275,13.330786,52.512864),
        "end_rect_coords": (13.328176,52.508454,13.328866,52.508568)
    },
    {
        "scenario_name": "Prinzenstr (Annenstr. to Moritzplatz)",
        "sumo_sim_files": ["prinzenstr_new_params.csv", "prinzenstr_default.csv"],
        "start_rect_coords": (13.413598,52.507573,13.414164,52.507822),
        "end_rect_coords": (13.410895,52.504089,13.411461,52.504338)
    },
    {
        "scenario_name": "Prinzenstr (Dresdener Str. to Sebastianstr.)",
        "sumo_sim_files": ["prinzenstr_new_params.csv", "prinzenstr_default.csv"],
        "start_rect_coords": (13.412877,52.506571,13.413298,52.506765),
        "end_rect_coords": (13.411595,52.504951,13.412016,52.505145)
    },
    {
        "scenario_name": "Oranienstr (Mariannenstr. to Adalbertstr.)",
        "sumo_sim_files": ["oranienstr_new_params.csv", "oranienstr_default.csv"],
        "start_rect_coords": (13.422222,52.500212,13.422505,52.500645),
        "end_rect_coords": (13.419491,52.500976,13.419774,52.501409)
    },
    {
        "scenario_name": "Oranienstr (Adalbertstr. to Oranienplatz)",
        "sumo_sim_files": ["oranienstr_new_params.csv", "oranienstr_default.csv"],
        "start_rect_coords": (13.418756,52.501182,13.419039,52.501615),
        "end_rect_coords": (13.416766,52.501724,13.417049,52.502157)
    },
    {
        "scenario_name": "Waldstr. (Turmstr. to Siemensstr.)",
        "sumo_sim_files": ["moabit_new_params.csv", "moabit_default.csv"],
        "start_rect_coords": (13.329907,52.527466,13.331534,52.527762),
        "end_rect_coords": (13.331344,52.532257,13.332392,52.532553)
    }
]

In [ ]:
duration_divergences = {}

for comp_data in comparison_data:
    scenario_name = comp_data.pop('scenario_name')

    print(f"----- {scenario_name} -----")
    duration_divergences[scenario_name] = analyze_and_plot_ride_data(get_ride_data(SUMO_SIM_DATA_FOLDER, **comp_data))

In [ ]:
duration_divergences

In [ ]:
divergences_default = []
divergences_new_params = []

for scenario_name, scenario_divergences in duration_divergences.items():
    for data_name, divergence in scenario_divergences.items():
        if data_name.endswith('_default'):
            divergences_default.append(divergence)
        elif data_name.endswith('_new_params'):
            divergences_new_params.append(divergence)

print(divergences_default)
print(divergences_new_params)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 16})
rc('text', usetex=True)
x_values = range(1, len(duration_divergences) + 1)

fig, (ax, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 6), gridspec_kw={'height_ratios': [5, 1]})
# fig, (ax, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 6))
ax.scatter(x_values, divergences_default, label="SUMO default parameters", color='green')
ax.scatter(x_values, divergences_new_params, label="SimRa-based re-parameterization", color='orange')

ax2.scatter(x_values, divergences_default, label="SUMO default parameters", color='green')
ax2.scatter(x_values, divergences_new_params, label="SimRa-based re-parameterization", color='orange')
ax2.set_ylim(bottom=0, top=0.1)

d = .015  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.xaxis.tick_top()
ax.tick_params(labeltop=False)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()

ax.set_title("Jensen–Shannon divergence beteween empirical distributions of ride durations for selected scenarios")
ax2.legend()

plt.savefig("duration_js_divergence.png", bbox_inches="tight")
plt.show()


In [ ]:
from brokenaxes import brokenaxes

fig = plt.figure(figsize=(14, 6))
bax = brokenaxes(ylims=((0, .1), (.4, .9)), hspace=.2)
bax.scatter(x_values, divergences_default, label="SUMO default parameters", color='green', s=80)
bax.scatter(x_values, divergences_new_params, label="SimRa-based re-parameterization", color='orange', s=80)

bax.legend(loc=3, frameon=True)
bax.set_xlabel('Scenario number')
bax.set_ylabel('JS divergence from SimRa data')
bax.set_title("Jensen–Shannon divergence beteween empirical distributions of ride durations for selected scenarios")

plt.savefig("duration_js_divergence.png", bbox_inches="tight")
plt.show()

In [ ]:

fig = plt.figure(figsize=(14, 6))

plt.bar(x_values, divergences_default, label="SUMO default parameters", color='green', align='edge', width=0.2)
plt.bar(x_values, divergences_new_params, label="SUMO re-parameterization", color='orange', align='edge', width=-0.2)

plt.xticks(x_values)
plt.legend(loc=3, frameon=True)
plt.xlabel('Scenario number')
plt.ylabel('JS divergence')
plt.title("Jensen–Shannon (JS) divergence\n of simulated ride durations from SimRa data")
# fig.suptitle("Jensen–Shannon (JS) divergence")

plt.savefig("duration_js_divergence.png", bbox_inches="tight")
plt.show()

In [ ]:
from pprint import pprint

scenario_number_mapping = {idx +1 : key for idx, key in enumerate(duration_divergences.keys())}

pprint(list(scenario_number_mapping.items()))